# Acronyms in the data and replacing

##### Get all acronyms in data and replace them with meaningful words

In [1]:
import nltk
import spacy
from spacy import displacy
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import datetime
import re
from collections import Counter
from itertools import chain
from nltk.corpus import stopwords
from textblob import TextBlob
%matplotlib inline
pd.options.display.max_columns = None

In [2]:
#load data
data = pd.read_csv('/home/haomiao/work/target0/data/initial_cleaned_text_Airsweb.csv')

/home/haomiao/python_venv/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (20,21,22,23,24,39,42,43,62,63,64,67,71,72,75,76,89,90) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#get the columns that contains acronyms to list
raw_remedial = data['Immediate remedial actions taken'].tolist()
raw_remedial.extend(data['What were you able to do about it?'].tolist())
raw_follow_up = data['Description of follow-up'].tolist()
raw_incident = data['Describe the incident:'].tolist()
raw_incident.extend(data['What did you see?'].tolist())
raw_incident.extend(data['What could have happened?'].tolist())
#remove nan
cleaned_raw_remedial = [x for x in raw_remedial if str(x) != 'nan']
cleaned_raw_follow_up = [x for x in raw_follow_up if str(x) != 'nan']
cleaned_raw_incident = [x for x in raw_incident if str(x) != 'nan']

In [4]:
acronyms_list = {'s&c':'S&C',
                 's&t':'S&T',
                 'h&s':'H&S',
                 'L&m':'L&M',
                 'H&s':'H&S',
                 'e&p':'E&P',
                 'h&S':'H&S'}

In [5]:
#replace lowercase into uppercase then replace the word 
def multipleReplace(text, wordDict=acronyms_list):
    for key in wordDict:
        text = text.replace(key, wordDict[key])
    return text
cleaned_raw_follow_up= [multipleReplace(text) for text in cleaned_raw_follow_up]
cleaned_raw_incident= [multipleReplace(text) for text in cleaned_raw_incident]
cleaned_raw_remedial= [multipleReplace(text) for text in cleaned_raw_remedial]


In [6]:
#get acronyms and count
acronyms = []
datalist = [cleaned_raw_follow_up, cleaned_raw_incident, cleaned_raw_remedial]
#pattern = r"[A-Z][a-zA-Z]*[A-Z]|[a-zA-Z]*\&[a-zA-Z]|(?:[A-Z]\.)+"
pattern = r"[a-zA-Z]*\&[a-zA-Z]*"
for data in datalist:
    for text in data:
        acronyms.extend(re.findall(pattern,text))
acronyms_count = pd.Series(Counter(acronyms)).sort_values(ascending = False)

In [ ]:
acronyms_df = acronyms_count.reset_index().assign(length = lambda x: x["index"].apply(len)).sort_values(0,ascending = False).pipe(lambda x: x[(x.length<6) & (x[0]>100)])

In [ ]:
acronyms_df.to_csv('acronyms_df.csv')

In [7]:
acronyms_count.index

Index(['&', 'S&C', 'S&T', 'H&S', 'E&P', 'E&S', 'A&E', 'D&A', 'S&CNA', 'L&M',
       'T&C', 'A&B', 'H&M', 'L&D', 'T&V', 'W&W', 'M&E', '&T', 'S&SD', 'R&M',
       'O&M', 'E&', 'J&A', 'R&B', 'A&D', 'N&M', 'B&B', 'P&C', 'R&C', 'D&P',
       'S&CAN', 'CS&TE', 'D&B', '&Cold', '&Environment', '&H', 'D&As', '&RS',
       'Consulting&Rail', 'Centre&', '&S', '&amp', 'D&G', '&b', '&collapse',
       'A&M', '&mud', '&out', '&run', '&the', '&tools', '&very', 'A&', 'CG&U',
       'work&spoke', 'D&UMAIN', 'W&B', 'up&dn', 'trip&', 'to&from', 'to&',
       'shape&', 'requirements&', 'post&wire', 'o&k', 'ft&', 'employees&',
       'ee&fcTime', 'called&', 'T&CP', 'Environment&Sustain', 'T&CE',
       'Siemans&', 'P&P', 'M&EE', 'Luton&Dunstable', 'K&H', 'IPS&E', 'H&SM',
       'warnings&regionName', 'H&E', 'H&', 'F&P', 'H&SAWA'],
      dtype='object')

In [ ]:
cleaned_raw_incident[0]

In [8]:
nlp = spacy.load('en')
doc = map(nlp,cleaned_raw_incident)
tokens_raw = [[(token.text,token.lemma_, token.tag_) for token in sent] for sent in doc]


In [9]:
words = list(chain(*tokens_raw))

In [10]:
verbs = {'VB',
         'VBD',
         'VBG',
         'VBN',
         'VBP',
         'VBZ' }
nouns = {'NN',
         'NNS',
         'NNP',
         'NNPS'}

In [ ]:
def retag_word(word):
    if word[2] in verbs:
        return (word[0],word[1],'V')
    elif word[2] in nouns:
        return (word[0],word[1],'N')
    else:
        return word
    
def retag_sentence(sentence):
    return [retag_word(word) for word in sentence]

In [ ]:
retagged_words = [retag_word(word) for word in words]

In [ ]:
#remove punct
punct=[",",".",":","(",")","-","/", " ","*","#"]
retagged_words = [i for i in retagged_words if i[0] not in punct ]
#remove stop words
filtered_words = [word for word in retagged_words if word[0] not in stopwords.words('english')]
#lowercase everything
filtered_words = [(word[0].lower(),word[1],word[2]) for word in filtered_words]

In [ ]:
verbs_nouns_incident = [i for i in filtered_words if i[2] in {"V","N"}]


In [ ]:
v_n_incident_df = pd.Series(Counter(word for word in verbs_nouns_incident)).sort_values(ascending = False).reset_index()
v_n_incident_df.rename(columns ={'level_0':'word','level_1':'word_lemma','level_2':'tag',0:'count'},inplace = True)

In [ ]:
verbs_incident = v_n_incident_df.loc[v_n_incident_df['tag'] == 'V'].sort(['count'],ascending = False,inplace=True)
nouns_incident = v_n_incident_df.loc[v_n_incident_df['tag'] == 'N'].sort(['count'],ascending = False,inplace = True)

In [ ]:
filtered_words

In [ ]:
pd.Series(Counter(word for word in filtered_words)).sort_values(ascending = False)

In [ ]:
tokens1

In [ ]:
#tagged_raw_incident = [TextBlob(' '.join(i)).tags for i in cleaned_raw_incident]
sent = TextBlob(cleaned_raw_incident[0]).tags
len(sent)

In [ ]:
tagged_raw_incident

In [ ]:
verbs = pd.read_csv('/home/haomiao/scratch/Target0/verbs.txt')

In [ ]:
verbs